In [1]:
import os
import csv
import matplotlib.pyplot as plt
import cv2
import easyocr
import numpy as np
import pandas as pandas

In [2]:
#ucitati sve slike iz tesnog skupa
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
    return images 

def load_sets():
    test_set = load_images_from_folder("skup_podataka/test")
    return test_set

def load_csv():
    file = open('skup_podataka/test_podaci.csv', encoding="utf8")
    csv_reader = csv.reader(file)
    header = []
    header = next(csv_reader)
    rows = []
    for row in csv_reader:
        rows.append(row)
    return rows

In [3]:
#ocr pomocu biblioteke
def ocr(imageset, csv_set):
    global_list = []
    reader = easyocr.Reader(['rs_latin'])
    
    len_of_image_with_books = 0
    
    index = 0 #index for csv_set
    for image in imageset:
        if csv_set[index][0] != '0':
            list_of_words = []

            #horizontalne knjige
            output = reader.readtext(image) #[lista_koordinata, rijec_koja_je_detektovana, koliko_je_siguran_da_je_to_ta_rijec]
            for o in output:
                if o[2]>0.5:
                    if(len(o[1]) > 1):
                        list_of_words.append(o[1])

            #odradi rotaciju za 90 stepeni da detektuje ukoliko ima vertikalnih knjiga
            rot_image = np.rot90(image)
            output = reader.readtext(rot_image) 
            for o in output:
                if o[2]>0.5:
                    if(len(o[1]) > 1):
                        list_of_words.append(o[1])

            global_list.append(list_of_words)
            
            index = index + 1
        else:
            empty_list = []
            global_list.append(empty_list)
            index = index + 1
        
    return global_list

In [23]:
def prepare_ocr_row(row):
    final_list = []
    for elem in row:
        mini_list = elem.split(" ")
        for word in mini_list:
            final_list.append(word.upper())
    return final_list

def prepare_csv_row(row):
    list1 = row.split("[")
    list2 = list1[1].split("]")
    return list2[0].split(";")

def calculate_result(ocr_set, csv_set):
    sum_of_all = 0
    num_of_words = 0
    
    for i in range(len(csv_set)):
        if(csv_set[i][0] != '0'):
            ocr_list = prepare_ocr_row(ocr_set[i])
            csv_list = prepare_csv_row(csv_set[i][1])
            
            num_of_words = num_of_words + len(csv_list)
            
            for word in csv_list:
                if word in ocr_list:
                    sum_of_all = sum_of_all + 1
    return (sum_of_all/num_of_words) 

In [5]:
#main
test_set = load_sets()
rows = load_csv()


In [10]:
list_of_detected_words = ocr(test_set, rows)
print(list_of_detected_words)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


[[], [], ['MRAČNA AKADEMIJA PODELJENE DUŠE', 'GABRIJELA PUL', 'DŽO', 'ABERKROMBI', 'ŠTRICA', 'Laguna', 'MRAČNA AKADEMIJA', 'KRVNE VEZE', 'GABRIJELA PUL'], [], ['ANITA DAJAMANT', 'DEVOJKA IZ BOSTONA', 'Laguna', 'SUZANA', 'DANLAP', 'LISTOV POLJUBAC', 'Lacuna', 'ANITA DAJAMANT', 'DOBRA LUKA', 'DVIVIVVI', 'DOBKV FOKV'], ['ALESANDRO BARIKO', 'SVILA', 'PAIDEIA', 'ALESANDRO BARIKO', 'PIJANISTA', 'PAIDEIA', 'ALESANDRO BARIKO', 'ZAMKOVI', 'PAIDEIA'], [], [], ['ALESANDRO BARIKO', 'SVILA', 'PAIDELA', 'PIJANISTA', 'PAIDEIA', 'ALESANDRO BARIKO'], [], ['freschello', 'ITALIA', 'emaker', 'KERI SMIT', 'Avulkan'], ['ČARLS', 'Stara prodavnica', 'DIKENS', 'retkosti'], [], [], ['GEORGE', 'NINA', 'Mali bistro', 'Bretanji'], [], ['Stiv Beri', 'ALEKSANDRIJSKA VEZA', 'vulkan', 'Stiv Beri', 'PAKLENA ODAJA', 'vulkan'], ['alnari', 'Sara', 'Šepard', 'Bezgrešna', 'alnari', 'malelažljivice', 'Sara Separd', 'Avulkan', 'Šeldon', 'SJAJ', 'pips', 'Slatke', 'ZVEZDA', 'Sidni'], [], ['KARLOS Ruis SAFON', 'PRINC', 'MAGLE', 

In [24]:
result = calculate_result(list_of_detected_words, rows)

In [29]:
procent = result * 100
round_procent = round(procent, 2)
print(str(round_procent) + "%")

82.39%
